# CAT BOOST Regressor

In [1]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold

In [3]:
# Load dataset
df = pd.read_csv('final_dataset_clusters-2.csv')

In [4]:
df

,country,updated_country,Country Code (ISO 3166-1 alpha-3),Year,gold,silver,bronze,total,medal_sum,check,...,BMI_obesity,BMI_morbid_obesity,Mean_Height,Diabetes_in_18+,Diabetes_treated_in_30+,non-HDL_cholesterol,HDL_cholesterol,Raised_blood_pressure,Hypertension,cluster
0,Afghanistan,Afghanistan,AFG,2000,0,0,0,0,0,True,...,0.037869,0.002605,159.842420,0.124577,0.227280,3.129567,1.128193,0.285826,0.360907,1
1,Afghanistan,Afghanistan,AFG,2004,0,0,0,0,0,True,...,0.053626,0.003623,160.101298,0.139517,0.230020,3.140379,1.120109,0.294444,0.367908,1
2,Afghanistan,Afghanistan,AFG,2008,0,0,1,1,1,True,...,0.072227,0.004871,160.345587,0.156097,0.231910,3.139214,1.110600,0.299683,0.375714,1
3,Afghanistan,Afghanistan,AFG,2012,0,0,1,1,1,True,...,0.095673,0.006511,160.547954,0.175056,0.233189,3.127356,1.098825,0.302909,0.383675,1
4,Algeria,Algeria,DZA,2000,1,1,3,5,5,True,...,0.131282,0.006170,164.568663,0.120538,0.384386,3.656587,1.040695,0.303167,0.375078,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,Vietnam,Vietnam,VNM,2012,0,0,0,0,0,True,...,0.007485,0.000263,160.226437,0.053830,0.308850,3.282843,1.100474,0.230051,0.287047,1
416,Zimbabwe,Zimbabwe,ZWE,2000,0,0,0,0,0,True,...,0.064462,0.004432,164.167920,0.097955,0.193180,2.811148,1.281791,0.295197,0.393215,1
417,Zimbabwe,Zimbabwe,ZWE,2004,1,1,1,3,3,True,...,0.072780,0.005343,164.048476,0.107090,0.198996,2.841984,1.288866,0.291489,0.407746,1
418,Zimbabwe,Zimbabwe,ZWE,2008,1,3,0,4,4,True,...,0.082211,0.006385,164.040913,0.110433,0.204597,2.856043,1.290640,0.287677,0.421400,1


# Dependent variables and Independent variables

In [5]:
df_numeric = df.drop(columns=["country", "updated_country", "Country Code (ISO 3166-1 alpha-3)","check","total","medal_sum","Year","cluster","gold","silver","bronze"])
X = df_numeric
y = df['total'] # Target Feature = Total medal count
X.shape, y.shape

((420, 19), (420,))

In [6]:
df_numeric

,GDP,HDI,LE,EYS,MYS,GNIPC,MMR,PHDI,CO2_PROD,BMI_underweight,BMI_obesity,BMI_morbid_obesity,Mean_Height,Diabetes_in_18+,Diabetes_treated_in_30+,non-HDL_cholesterol,HDL_cholesterol,Raised_blood_pressure,Hypertension
0,594.630107,0.33050,53.66650,5.126431,1.176271,1289.174029,1389.824073,0.32750,0.069803,0.215176,0.037869,0.002605,159.842420,0.124577,0.227280,3.129567,1.128193,0.285826,0.360907
1,446.613504,0.35775,56.22350,6.294416,1.341300,1214.533525,1273.243525,0.35475,0.060152,0.179564,0.053626,0.003623,160.101298,0.139517,0.230020,3.140379,1.120109,0.294444,0.367908
2,505.708140,0.40825,58.52500,7.986830,1.563972,1608.671663,1071.398197,0.40525,0.080722,0.148625,0.072227,0.004871,160.345587,0.156097,0.231910,3.139214,1.110600,0.299683,0.375714
3,690.284444,0.44450,60.62150,9.043315,1.846628,1964.609019,914.435116,0.44100,0.273967,0.119533,0.095673,0.006511,160.547954,0.175056,0.233189,3.127356,1.098825,0.302909,0.383675
4,2312.433986,0.63075,69.35075,10.553575,6.150535,8086.780317,186.703561,0.60550,3.255793,0.074711,0.131282,0.006170,164.568663,0.120538,0.384386,3.656587,1.040695,0.303167,0.375078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,1698.498729,0.67325,73.52850,12.487576,7.554262,6005.400737,85.813036,0.64600,1.524574,0.205790,0.007485,0.000263,160.226437,0.053830,0.308850,3.282843,1.100474,0.230051,0.287047
416,18086.935704,0.44075,47.24325,10.101884,6.162323,1625.965635,347.787638,0.43150,1.281039,0.108578,0.064462,0.004432,164.167920,0.097955,0.193180,2.811148,1.281791,0.295197,0.393215
417,15718.856494,0.42075,43.65025,10.538570,6.764291,1376.309528,488.384600,0.41225,1.022238,0.101842,0.072780,0.005343,164.048476,0.107090,0.198996,2.841984,1.288866,0.291489,0.407746
418,10949.861255,0.42100,45.06100,10.470320,7.209539,1114.551383,558.936421,0.41350,0.820263,0.094166,0.082211,0.006385,164.040913,0.110433,0.204597,2.856043,1.290640,0.287677,0.421400


# Scaling

In [7]:
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
print(X_scaled)
print(X_scaled.min())
print(X_scaled.max())

[[-3.64725472e-01 -2.37522506e+00 -2.33040872e+00 ... -3.46580920e-01
   4.19657670e-01 -2.24362220e-01]
 [-3.70425624e-01 -2.21822110e+00 -2.02387143e+00 ... -3.76967881e-01
   5.62760619e-01 -1.39792970e-01]
 [-3.68149876e-01 -1.92725963e+00 -1.74796389e+00 ... -4.12711829e-01
   6.49744544e-01 -4.55125166e-02]
 ...
 [ 2.17711762e-01 -1.85523947e+00 -3.53117297e+00 ...  2.57371802e-01
   5.13686760e-01  3.41414169e-01]
 [ 3.40567222e-02 -1.85379906e+00 -3.36204998e+00 ...  2.64040793e-01
   4.50389463e-01  5.06342502e-01]
 [ 9.64040627e-04 -1.60460929e+00 -2.80636871e+00 ...  2.46579206e-01
   3.84928104e-01  5.70079763e-01]]
-3.956611570247938
59.22588817796017


In [9]:
y.max()

317

# K fold cross validation
Model used : Catboost
RMSE AND R2 score

In [23]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
rf = CatBoostRegressor(n_estimators = 500, random_state = 42)
rmse_scores = []
r2_scores = []
rmse_train_scores = []
r2_train_scores = []
for train_index, test_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)

    train_pred = rf.predict(X_train)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    rmse_train = np.sqrt(mean_squared_error(y_train, train_pred))
    r2_train = r2_score(y_train, train_pred)

    rmse_train_scores.append(rmse_train)
    r2_train_scores.append(r2_train)

    rmse_scores.append(rmse)
    r2_scores.append(r2)


print("Training Results")
print(f"Average RMSE: {np.mean(rmse_train_scores)}")
print(f"Average R2: {np.mean(r2_train_scores)}")
print('-------------------------------')
print("Testing Results")
print(f"Average RMSE: {np.mean(rmse_scores)}")
print(f"Average R2: {np.mean(r2_scores)}")

Learning rate set to 0.060545
0:	learn: 38.9624743	total: 11.7ms	remaining: 5.86s
1:	learn: 38.2686896	total: 21.3ms	remaining: 5.3s
2:	learn: 37.4966434	total: 35.9ms	remaining: 5.94s
3:	learn: 36.8958315	total: 45.3ms	remaining: 5.62s
4:	learn: 36.3580178	total: 57.6ms	remaining: 5.71s
5:	learn: 35.8338662	total: 69.9ms	remaining: 5.76s
6:	learn: 35.1656532	total: 80.9ms	remaining: 5.7s
7:	learn: 34.6387609	total: 96.3ms	remaining: 5.92s
8:	learn: 34.2661632	total: 108ms	remaining: 5.88s
9:	learn: 33.8482510	total: 119ms	remaining: 5.84s
10:	learn: 33.3222816	total: 130ms	remaining: 5.76s
11:	learn: 32.8086120	total: 139ms	remaining: 5.65s
12:	learn: 32.3280485	total: 148ms	remaining: 5.56s
13:	learn: 31.8877317	total: 159ms	remaining: 5.54s
14:	learn: 31.3533560	total: 170ms	remaining: 5.49s
15:	learn: 30.8702497	total: 181ms	remaining: 5.48s
16:	learn: 30.5949104	total: 193ms	remaining: 5.49s
17:	learn: 30.2567805	total: 212ms	remaining: 5.68s
18:	learn: 29.8650484	total: 239ms	rem

In [24]:
print(f"Min: {y.min()}")
print(f"Max: {y.max()}")
print(f"Mean value of y: {y.mean()}")
print(f"Standard deviation of y: {y.std()}")

Min: 0
Max: 317
Mean value of y: 18.85952380952381
Standard deviation of y: 40.371860752065395
